In [1]:
from congress import Congress
api_key = 'fxzKX1XVwgCBfElNZlya7fwBVdEFNBlGEadiAFxD'
congress_api = Congress(api_key)
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
import csv
import json
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1.5)
import numpy as np
import importlib
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import CleanData, SaveState
importlib.reload(CleanData)
importlib.reload(SaveState)
from CleanData import clean_comment, get_bag_of_words

[nltk_data] Downloading package stopwords to /home/zchao3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Get stopword list
stop_words = stopwords.words('english')
number_strs = ['one', 'two', 'three', 'four', 'five', 'six', 'seven',\
'eight', 'nine', 'ten']
stop_words.extend(number_strs)

In [6]:
# Load tweet from members
filename = '../data/115_Senators_during_term.csv'
tweets_df = pd.read_csv(filename)
# tweets_df = tweets_df.head(200000)
tweets_df.head(10)

,version https://git-lfs.github.com/spec/v1
0,oid sha256:928a636f187541025f3a1045d8df85ed0c9c9ff7563d68ed5ea752d5018da930
1,size 61075637


In [4]:
tweets_df = tweets_df.loc[tweets_df['full_text'].apply(type) == str]

KeyError: 'full_text'

In [ ]:
tweets_df['user_name'].unique()

In [ ]:
dem_train_users = ['Chuck Schumer', 'Elizabeth Warren', 'Bernie Sanders','Michael Bennet', \
                   'Cory Booker', 'Amy Klobuchar']
rep_train_users = ['McConnell', 'Rubio', 'John McCain', 'Ted Cruz']

In [ ]:
test_users = tweets_df['user_name'].unique().tolist()
for user in (dem_train_users + rep_train_users):
    test_users.remove(user)

In [ ]:
print('No. of democratic training tweets is',
tweets_df.loc[tweets_df['user_name'].isin(dem_train_users)].shape[0])
print('No. of republican training tweets is',
tweets_df.loc[tweets_df['user_name'].isin(rep_train_users)].shape[0])

In [ ]:
print('No. of testing tweets is',
tweets_df.loc[tweets_df['user_name'].isin(test_users)].shape[0])

In [ ]:
print(tweets_df.columns)

In [ ]:
tweets_df['clean_tweet'] = tweets_df['full_text'].apply(clean_comment)
bag_of_words, vectorizer = get_bag_of_words(tweets_df['clean_tweet'],ngram_range=(1,3), min_df=0.001)
print(bag_of_words.shape) 

In [ ]:
tweets_df.head(5)

In [ ]:
# Train binary multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn import metrics
def get_binary_NB_model(bag_of_words, df):
    # Training data:
    class1_words = bag_of_words[df['user_name'].isin(dem_train_users),:]
    class2_words = bag_of_words[df['user_name'].isin(rep_train_users),:]
    train_tweets = np.concatenate((class1_words,class2_words))
    labels = np.concatenate((np.zeros(class1_words.shape[0]),np.ones(class2_words.shape[0])))
    nb = ComplementNB()
    nb.fit(train_tweets, labels)
    # # Performance on training data
    predictions = nb.predict(train_tweets)
    print('Training Accuracy: ' + str(sum(labels==predictions)/len(labels)))
    # Compute the error.
    tn, fp, fn, tp = metrics.confusion_matrix(labels,predictions).ravel()
    print(tn, fp, fn, tp)
    return nb

In [ ]:
# Train a naive bayes model on tweets from specified training users from each party.
nb_model = get_binary_NB_model(bag_of_words, tweets_df)